对第十个视频预测方向盘转角，并生成可视化展示视频。  
第十个视频文件名`epoch10_front.mkv`

Steps:
* load model
* slice video to images
* get steering predicitons
* visualizaiton （call `utils.visualize`）

In [1]:
#!/usr/bin/env python 
import sys
import os
import time
import subprocess as sp
import itertools
import cv2
import numpy as np
import tensorflow as tf
import utils
from utils import visualize
import params 
import imageio

%load_ext autoreload
%autoreload 2


Using TensorFlow backend.


In [4]:
##Load model
model = utils.get_model()


Model already exists, do you want to reuse? (y/n): y
Model fetched from the disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 98, 24)        96        
_________________________________________________________________
activation_1 (Activation)    (None, 31, 98, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 47, 36)        144       
_____________________________________________________________

In [2]:
##  image Preprocess
def img_pre_process(img, YUV=True):
    """
    Processes the image and returns it
    :param img: The image to be processed
    :YUV=Ture, turn RGB to YUV, else keep RGB
    :return: Returns the processed image(crop, resize, YUV, normalization)
    """
    # Chop off 1/3 from the top and cut bottom 150px(which contains the head of car) img[240:570,:,:]
    shape = img.shape
    img_crop = img[int(shape[0]/3):shape[0]-150, 0:shape[1]]
    
    # Resize the image
    img_resize = cv2.resize(img_crop, (200, 66), interpolation=cv2.INTER_AREA)
    
    if YUV:
        # RGB to YUV
        img_yuv = cv2.cvtColor(img_resize, cv2.COLOR_BGR2YUV)

        # Normalize, need int???
        img_nor = img_yuv/255.0
    else:
        img_nor = img_resize/255.0
    assert img_nor.shape == (66,200,3)
    assert img_nor.min() >= 0
    assert img_nor.max() <= 1
    
    return img_nor


In [5]:

## Process video
epoch_id = 10
print('---------- processing video for epoch {} ----------'.format(epoch_id))

   
# convert mkv to mp4
vid_path = utils.join_dir(params.data_dir, 'epoch{:0>2}_front.mkv'.format(epoch_id))
vid_mp4_path = utils.join_dir(params.data_dir, 'epoch{:0>2}_front.mp4'.format(epoch_id))
if not os.path.exists(vid_mp4_path):
    mkv_to_mp4(vid_mkv_path, remove_mkv=False)    

# slice mp4 to images
vid = imageio.get_reader(vid_mp4_path,'ffmpeg')
machine_steering = []
frame_count = len(vid)

## Steering Predictions
print('performing inference...')
time_start = time.time()
for num in range(len(vid)):  
    img = vid.get_data(num)
    if img is not None:
        img = img_pre_process(img)
        assert img.shape==(66,200,3)
        ste = float(model.predict(img.reshape(-1,*img.shape), batch_size=1))
        machine_steering.append(ste)


fps = frame_count / (time.time() - time_start)

print('completed inference, total frames: {}, average fps: {} Hz'.format(frame_count, round(fps, 2)))



---------- processing video for epoch 10 ----------
performing inference...
completed inference, total frames: 2700, average fps: 80.35 Hz


预处理2700张图像，用时15.76s，平均一张图像用时5.84ms;   
预测2700图像22.04s，平均一张图像用时8.16ms;  
预处理+预测 共用时37.8s，平均一张图像用时14ms, 也就是说每秒能处理71.4张图像  

In [6]:
## visualization
print('performing visualization...')
utils.visualize(epoch_id, machine_steering, params.out_dir,
                    verbose=True, frame_count_limit=None)



performing visualization...
completed 100 of 2700 frames
completed 200 of 2700 frames
completed 300 of 2700 frames
completed 400 of 2700 frames
completed 500 of 2700 frames
completed 600 of 2700 frames
completed 700 of 2700 frames
completed 800 of 2700 frames
completed 900 of 2700 frames
completed 1000 of 2700 frames
completed 1100 of 2700 frames
completed 1200 of 2700 frames
completed 1300 of 2700 frames
completed 1400 of 2700 frames
completed 1500 of 2700 frames
completed 1600 of 2700 frames
completed 1700 of 2700 frames
completed 1800 of 2700 frames
completed 1900 of 2700 frames
completed 2000 of 2700 frames
completed 2100 of 2700 frames
completed 2200 of 2700 frames
completed 2300 of 2700 frames
completed 2400 of 2700 frames
completed 2500 of 2700 frames
completed 2600 of 2700 frames
